In [1]:
import os, gc, cv2, torch, h5py, shutil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
from tqdm import tqdm
from transformers import AutoImageProcessor, ZoeDepthForDepthEstimation

# ========== 设备 ==========
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ========== 模型 ==========
model_name = "Intel/zoedepth-nyu-kitti"
processor = AutoImageProcessor.from_pretrained(model_name)
model = ZoeDepthForDepthEstimation.from_pretrained(model_name).to(device)
model.eval()

# ========== 数据路径 ==========
mat_path = '/kaggle/input/nyuv2-dataset/nyu_depth_v2_labeled.mat'
idx_list_path = '/kaggle/input/nyu2-test/nyu2_test.txt'
save_root = '/kaggle/working'
os.makedirs(save_root, exist_ok=True)
vis_dir = os.path.join(save_root, 'visualizations_zoe_nyu')
os.makedirs(vis_dir, exist_ok=True)

# ========== 读 NYUv2 ==========
with h5py.File(mat_path, 'r') as f:
    images = f['images'][:]
    depths = f['depths'][:]

images = np.transpose(images, (3, 2, 1, 0))
depths = np.transpose(depths, (2, 1, 0))

idx_list = np.loadtxt(idx_list_path).astype(int) - 1

# ========== 工具函数 ==========
def save_vis(rgb, gt, pred, mask, save_path):
    plt.figure(figsize=(15, 5))
    valid = mask & (gt > 0) & np.isfinite(gt) & np.isfinite(pred)
    if np.sum(valid) == 0:
        vmin, vmax = 0, 1
    else:
        all_depths = np.concatenate([gt[valid], pred[valid]])
        vmin = np.percentile(all_depths, 2)
        vmax = np.percentile(all_depths, 98)

    plt.subplot(1, 3, 1)
    plt.imshow(rgb)
    plt.title("RGB"); plt.axis("off")

    plt.subplot(1, 3, 2)
    plt.imshow(gt, cmap='viridis', vmin=vmin, vmax=vmax)
    plt.title("GT Depth"); plt.colorbar(fraction=0.046, pad=0.04); plt.axis("off")

    plt.subplot(1, 3, 3)
    plt.imshow(pred, cmap='viridis', vmin=vmin, vmax=vmax)
    plt.title("Predicted (Aligned)"); plt.colorbar(fraction=0.046, pad=0.04); plt.axis("off")

    plt.tight_layout()
    plt.savefig(save_path, dpi=150)
    plt.close()

def compute_abs_rel(pred, gt, mask):
    valid = mask & (gt > 0)
    pred, gt = pred[valid], gt[valid]
    return np.mean(np.abs(pred - gt) / gt) if len(gt) > 0 else np.nan

def compute_delta1(pred, gt, mask):
    valid = mask & (gt > 1e-6) & (pred > 1e-6)
    pred, gt = pred[valid], gt[valid]
    thresh = np.maximum(pred / gt, gt / pred)
    return np.mean(thresh < 1.25) if len(thresh) > 0 else np.nan

def scale_match(pred, gt, mask):
    pred, gt = pred[mask], gt[mask]
    denom = np.sum(pred ** 2)
    return (np.sum(gt * pred) / denom) if denom > 1e-8 else 1.0

# ========== 主评测 ==========
results = []

for idx in tqdm(idx_list, desc="Evaluating"):
    try:
        # 取出数据
        rgb = images[:, :, :, idx].astype(np.uint8)      # (480, 640, 3)
        gt_depth = depths[:, :, idx].astype(np.float32)  # (480, 640), 单位米
        mask = (gt_depth > 0) & np.isfinite(gt_depth)

        # 前处理
        input_image = Image.fromarray(rgb)
        inputs = processor(images=input_image, return_tensors="pt").to(device)

        # 前向
        with torch.no_grad():
            outputs = model(**inputs)

        # 后处理到原图尺寸 (H, W)
        H, W = rgb.shape[0], rgb.shape[1]
        processed = processor.post_process_depth_estimation(
            outputs, source_sizes=[(H, W)]
        )
        pred_depth = processed[0]["predicted_depth"].squeeze().detach().cpu().numpy()

        # 尺寸对齐
        if pred_depth.shape != gt_depth.shape:
            pred_depth = cv2.resize(pred_depth, (gt_depth.shape[1], gt_depth.shape[0]),
                                    interpolation=cv2.INTER_LINEAR)

        # 缩放匹配 + 指标
        scale = scale_match(pred_depth, gt_depth, mask)
        pred_aligned = pred_depth * scale
        abs_rel = compute_abs_rel(pred_aligned, gt_depth, mask)
        delta1 = compute_delta1(pred_aligned, gt_depth, mask)
        results.append([int(idx), float(abs_rel), float(delta1)])

        print(f"[{idx}] AbsRel: {abs_rel:.4f}, δ1: {delta1*100:.2f}%")

        # 可视化
        save_path = os.path.join(vis_dir, f"{idx:04d}_zoe_nyu.png")
        save_vis(rgb, gt_depth, pred_aligned, mask, save_path)

        # 清理
        del rgb, gt_depth, pred_depth, pred_aligned, inputs, outputs, processed
        torch.cuda.empty_cache(); 
        gc.collect()

    except Exception as e:
        print(f"跳过 {idx}，错误: {e}")
        continue

# ========== 保存结果 ==========
df = pd.DataFrame(results, columns=["idx", "AbsRel", "Delta1"])
mean_absrel = df["AbsRel"].mean()
mean_delta1 = df["Delta1"].mean()
df = pd.concat([df, pd.DataFrame([["mean", mean_absrel, mean_delta1]], columns=["idx", "AbsRel", "Delta1"])],
               ignore_index=True)

csv_path = os.path.join(save_root, "zoe_nyuv2_results.csv")
df.to_csv(csv_path, index=False)
print(f"评估结果已保存：{csv_path}")

# 打包可视化
zip_path = shutil.make_archive(os.path.join(save_root, "zoe_nyuv2_visualizations"), 'zip', root_dir=vis_dir)
print(f"可视化图像已打包: {zip_path}")


2025-08-08 14:52:59.524946: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754664779.744397      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754664779.807897      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


preprocessor_config.json:   0%|          | 0.00/723 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.38G [00:00<?, ?B/s]

...

[1444] AbsRel: 0.1996, δ1: 64.19%


Evaluating: 100%|█████████▉| 665/666 [30:58<00:04,  4.17s/it]

[1447] AbsRel: 0.1537, δ1: 83.60%


Evaluating: 100%|██████████| 666/666 [31:02<00:00,  2.80s/it]


评估结果已保存：/kaggle/working/zoe_nyuv2_results.csv
可视化图像已打包: /kaggle/working/zoe_nyuv2_visualizations.zip
